In [4]:
import numpy as np
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
torch.backends.mps.is_built()

In [ ]:
import torch.nn as nn


class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding='same', bias=False),
            nn.ELU(),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding='same', bias=False),
            nn.BatchNorm2d(128),
            nn.ELU(),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding='same', bias=False),
            nn.BatchNorm2d(256),
            nn.ELU(),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding='same', bias=False),
            nn.BatchNorm2d(512),
            nn.ELU(),
            nn.Conv2d(in_channels=512, out_channels=1, kernel_size=1, bias=False)
        )
        self.act = nn.Sigmoid()

    def forward(self, input):
        out = self.classifier(input)
        out = out.view((input.shape[0], -1))
        out = self.act(out)
        return out


class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.gen = nn.Sequential(
            nn.ConvTranspose2d(in_channels=36, out_channels=512, kernel_size=4, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=4, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=4, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=128, out_channels=1, kernel_size=4, bias=False),
            nn.Tanh()
        )
        self.output_linear = nn.Linear(in_features=169, out_features=150)

    def forward(self, input):
        input = input.unsqueeze(-1).unsqueeze(-1)
        out = self.gen(input)
        out = out.view((-1, 169))
        out = self.output_linear(out)
        out = out.view((-1, 1, 10, 15))
        return out

In [4]:
from data.preprocessing import read_csv_2d

data, features = read_csv_2d(filename='data/data_v4/csv/digits.csv', pad_range=(-3, 5), time_range=(-7, 9), strict=False)
data.shape

(20000, 8, 16)

In [12]:
def preprocess_features_v4plus(features):
    # features:
    #   crossing_angle [-20, 20]
    #   dip_angle [-60, 60]
    #   drift_length [35, 290]
    #   pad_coordinate [40-something, 40-something]
    #   padrow {23, 33}
    #   pT [0, 2.5]
    bin_fractions = torch.tensor(features[:, 2:4] % 1)
    features_1 = (torch.tensor(features[:, :3]) - torch.tensor([[0.0, 0.0, 162.5]])) / torch.tensor([[20.0, 60.0, 127.5]])
    features_2 = torch.tensor(features[:, 4:5] >= 27, dtype=torch.float32)
    features_3 = torch.tensor(features[:, 5:6] / 2.5)
    return torch.cat([features_1, features_2, features_3, bin_fractions], axis=-1)

In [14]:
import torch

preprocess_features_v4plus(features).shape

torch.Size([20000, 5])

In [ ]:
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.gen = nn.Sequential(
            nn.ConvTranspose2d(in_channels=3, out_channels=32, kernel_size=4, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, bias=False),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=64, out_channels=128, kernel_size=4, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, bias=False),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=4, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=16, kernel_size=3, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, bias=False),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=16, out_channels=1, kernel_size=4, bias=False)
        )
        self.input_linear = nn.Conv2d(in_channels=32, out_channels=480, kernel_size=1, bias=False)
        self.output_linear = nn.ConvTranspose2d(in_channels=160, out_channels=150, kernel_size=1, bias=False)
        # self.apply(weights_init)

    def forward(self, x):
        x = x.unsqueeze(-1).unsqueeze(-1)
        out = self.input_linear(x)
        out = out.view((-1, 3, 4, 40))
        out = self.gen(out)
        out = out.view((-1, 160, 1, 1))
        out = self.output_linear(out)
        out = out.view((-1, 1, 10, 15))
        return out

In [ ]:
import torch

x = torch.rand((64, 32))

g = Generator()
g(x)

In [6]:
import yaml
from nn import BuildModel

file = './model/baseline_8x16_GP50_full_fc.yaml'

with open(file, 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

architecture_descr = config['architecture']

discriminator = BuildModel(architecture_descr['discriminator'])

8


In [10]:
import torch
x = torch.rand((32, 1, 8, 16))
features = torch.rand((32, 7))
batch = [features, x]
discriminator(batch).shape

torch.Size([32, 8, 8, 16])
ConvBlock(
  (layers): ModuleList(
    (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ELU(alpha=1.0)
    (2): Dropout(p=0.02, inplace=False)
    (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (4): ELU(alpha=1.0)
    (5): Dropout(p=0.02, inplace=False)
    (6): MaxPool2d(kernel_size=[1, 2], stride=[1, 2], padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (8): ELU(alpha=1.0)
    (9): Dropout(p=0.02, inplace=False)
    (10): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (11): ELU(alpha=1.0)
    (12): Dropout(p=0.02, inplace=False)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=valid)
    (15): ELU(alpha=1.0)
    (16): Dropout(p=0.02, inplace=False)
    (17): Conv2d(64, 64, kernel_size=(2, 2), stride=(1, 1), padding=

torch.Size([32, 1])

In [9]:
discriminator.state_dict()

OrderedDict([('models.0.block.layers.0.weight',
              tensor([[[[ 0.1141, -0.0067, -0.0813],
                        [-0.0099,  0.0580, -0.0253],
                        [ 0.0399,  0.0733,  0.0577]],
              
                       [[ 0.0051, -0.0274,  0.0043],
                        [-0.0194,  0.0147, -0.0512],
                        [ 0.1170, -0.1103,  0.1063]],
              
                       [[-0.0877,  0.0716, -0.0983],
                        [-0.0109,  0.0315,  0.0927],
                        [ 0.0592, -0.0118,  0.0328]],
              
                       ...,
              
                       [[-0.0105,  0.0425, -0.0776],
                        [ 0.0581, -0.1102, -0.0714],
                        [-0.0616, -0.0245,  0.1016]],
              
                       [[-0.0360,  0.0257,  0.0918],
                        [-0.0523, -0.0332, -0.0394],
                        [ 0.1070,  0.0989, -0.0313]],
              
                       [[ 0.0799, 

In [1]:
from metrics import get_val_metric_v, _METRIC_NAMES
from data import preprocessing
import matplotlib.pyplot as plt

pad_range = (-3, 5)
time_range = (-7, 9)
data, features = preprocessing.read_csv_2d(
    pad_range=pad_range, time_range=time_range
)

metric = get_val_metric_v(data)

/Users/karim/Desktop/tpc-gen-torch/data/data_v4plus_shortFixedPt/csv/digits.csv


In [ ]:
features.shape

In [7]:
from utils import LoadData
import torchvision.transforms as T

transform = T.Compose([
    T.ToTensor()
])
dataset = LoadData([data, features], transform=transform)

In [9]:
from torch.utils.data import DataLoader

loader = DataLoader(dataset, batch_size=32, num_workers=4, pin_memory=True)


In [28]:
import numpy as np
x = features[: , 3]

bins = np.linspace(np.min(x), np.max(x), 101)

(x[:, np.newaxis] < bins[np.newaxis, 1:]).argmax(1)

array([27, 36, 37, ..., 30, 80, 10])

In [5]:
!python main.py

/Users/karim/opt/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/karim/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  Referenced from: <607DE3C1-0940-3CDA-A725-73ADE160D092> /Users/karim/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so
  Expected in:     <9D4C7FD2-49A8-383A-AC3E-A560DE81B0D8> /Users/karim/opt/anaconda3/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")

----------------------------------------
Arguments:
    config : None
    checkpoint_name : None
    gpu_num : None
    prediction_only : False
    logging_dir : logs
----------------------------------------

cpu
None
data/data_v4plus/csv/digits.csv
